In [6]:
from __future__ import (absolute_import, division, print_function,unicode_literals)

import yfinance as yf
import pandas as pd
import backtrader as bt
from datetime import datetime
import os
import sys
import json
from backtrader.analyzers import Returns,DrawDown,SharpeRatio,TradeAnalyzer

In [2]:
# import strategies
sys.path.append('../strategies/')
from  Test_strategy import TestStrategy
from sma import SMA
from sma_rsi import SMA_RSI

In [3]:
def prepare_cerebro(asset,strategy,data_path,start_date:str,end_date:str=datetime.now(),cash:int=100,commission:float=0)->bt.Cerebro:
    if end_date ==None:
        end_date= datetime.strftime(datetime.now(),"%Y-%m-%d")

    cerebro = bt.Cerebro()
    cerebro.broker.setcash(cash)
    cerebro.broker.setcommission(commission=commission)
    cerebro.addstrategy(strategy)
    
    isExist = os.path.exists(data_path)
    if not isExist:
        data= yf.download(asset,start_date,end=end_date)
        data.to_csv(data_path)
    
    data = bt.feeds.YahooFinanceCSVData(
        dataname=data_path,
        fromdate=datetime.strptime(start_date,"%Y-%m-%d"),
        todate=datetime.strptime(end_date,"%Y-%m-%d"),
        reverse=False
    )

    cerebro.adddata(data)
    # cerebro.addanalyzer(AnnualReturn)
    cerebro.addanalyzer(TradeAnalyzer)
    return cerebro

In [4]:
def run_test(cerebro:bt.Cerebro):
    
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    res=cerebro.run()

    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

    return res

In [17]:
cerebro=prepare_cerebro('BTC',SMA,"btc.csv","2021-10-01","2022-10-19")
# Analyzer
cerebro.addanalyzer(SharpeRatio, _name='sharpe')
cerebro.addanalyzer(Returns, _name='returns')
cerebro.addanalyzer(DrawDown, _name='draw')
cerebro.addanalyzer(TradeAnalyzer, _name='trade')
thestrats=run_test(cerebro)
thestrat = thestrats[0]

sharpe=thestrat.analyzers.sharpe.get_analysis()
print('Sharpe Ratio:',sharpe['sharperatio'] )

return_val=thestrat.analyzers.returns.get_analysis()
print('Return:', return_val['rtot'])

drawdown=thestrat.analyzers.draw.get_analysis()
print('max draw:', drawdown['max']['drawdown'])

trade=thestrat.analyzers.trade.get_analysis()
print('win trade:', trade['won']['total'])
print('loss trade:', trade['lost']['total'])
print('number trade:', trade['total']['total'])

# print(json.dumps(thestrat.analyzers.trade.get_analysis(),indent=4))
# print(json.dumps(parsed, indent=4))

Starting Portfolio Value: 100.00
Final Portfolio Value: 98.09
Sharpe Ratio: -3.119131412713244
Return: -0.019284761411866646
max draw: 2.8770532603284797
win trade: 7
loss trade: 8
number trade: 15


In [ ]:
cerebro.addwriter(bt.WriterFile, csv="-wcsv", rounding=4)

In [ ]:
cerebro=prepare_cerebro('BTC',SMA,"btc.csv","2021-10-01","2022-10-19")
run_test(cerebro)

In [ ]:
cerebro=prepare_cerebro('BTC',SMA,"btc.csv","2021-10-01","2022-10-19")
run_test(cerebro)